In [2]:
from scipy.stats import norm,t
from scipy.integrate import quad
import pandas as pd
import numpy as np
'''
Calculate Return
'''

# Implement the function to calculate the return
def return_calculate(prices, method='ARS', dateColumn='Date'):
    # Exclude the date column from the calculations
    tickers = [col for col in prices.columns if col != dateColumn]
    df = prices[tickers] # The dataframe is now with no date column.
    
    # Calculate the return using Classical Brownian Motion.
    if method == 'CBM':
        df = df.diff().dropna()
    
    # Calculate the return using Arithmetic Return System.
    elif method == 'ARS':
        df = (df - df.shift(1)) / df.shift(1)
        df = df.dropna()
        
    # Calculate the return using Geometric Brownian Motion.
    elif method == 'GBM':
        df = np.log(df).diff().dropna()
        
    else:
        raise ValueError(f"method: {method} must be in (\"CBM\",\"ARS\",\"GBM\")")
    
    return df

'''
Multivariate PCA Simulation
'''
def simulatePCA(N, df, mean=None, seed=1234, pctExp=1):
    # Error Checking
    m, n = df.shape
    if n != m:
        raise ValueError(f"Covariance Matrix is not square ({n},{m})")
    
    # Initialize the output
    out = np.zeros((N, n))
    
    # Set mean
    if mean is None:
        mean = np.zeros(n)
    else:
        if len(mean) != n:
            raise ValueError(f"Mean ({len(mean)}) is not the size of cov ({n},{n})")
    
    eigenvalues, eigenvectors = np.linalg.eig(df)
    
    # Get the indices that would sort eigenvalues in descending order
    indices = np.argsort(eigenvalues)[::-1]
    # Sort eigenvalues
    eigenvalues = eigenvalues[indices]
    # Sort eigenvectors according to the same order
    eigenvectors = eigenvectors[:, indices]
    
    tv = np.sum(eigenvalues)
    posv = np.where(eigenvalues >= 1e-8)[0]
    if pctExp <= 1:
        nval = 0
        pct = 0.0
        # How many factors needed
        for i in posv:
            pct += eigenvalues[i] / tv
            nval += 1
            if pct >= pctExp:
                break
    
     # If nval is less than the number of positive eigenvalues, truncate posv
    if nval < len(posv):
        posv = posv[:nval]
        
    # Filter eigenvalues based on posv
    eigenvalues = eigenvalues[posv]
    eigenvectors = eigenvectors[:, posv]
    
    B = eigenvectors @ np.diag(np.sqrt(eigenvalues))
    
    np.random.seed(seed)
    rand_normals = np.random.normal(0.0, 1.0, size=(N, len(posv)))
    out = np.dot(rand_normals, B.T) + mean
    
    return out.T

'''
VaR/ES on 2 levels from simulated values - Copula
'''

def simulateCopula(portfolio, returns):
    portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']
    models = {}
    uniform = pd.DataFrame()
    standard_normal = pd.DataFrame()
    
    for stock in portfolio["Stock"]:
        # If the distribution for the model is normal, fit the data with normal distribution.
        if portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'Normal':
            models[stock] = norm.fit(returns[stock])
            mu, sigma = norm.fit(returns[stock])
            
            # Transform the observation vector into a uniform vector using CDF.
            uniform[stock] = norm.cdf(returns[stock], loc=mu, scale=sigma)
            
            # Transform the uniform vector into a Standard Normal vector usig the normal quantile function.
            standard_normal[stock] = norm.ppf(uniform[stock])
            
        # If the distribution for the model is t, fit the data with normal t.
        elif portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'T':
            models[stock] = t.fit(returns[stock])
            nu, mu, sigma = t.fit(returns[stock])
            
            # Transform the observation vector into a uniform vector using CDF.
            uniform[stock] = t.cdf(returns[stock], df=nu, loc=mu, scale=sigma)
            
            # Transform the uniform vector into a Standard Normal vector usig the normal quantile function.
            standard_normal[stock] = norm.ppf(uniform[stock])
        
    # Calculate Spearman's correlation matrix
    spearman_corr_matrix = standard_normal.corr(method='spearman')
    
    nSim = 10000
    
    # Use the PCA to simulate the multivariate normal.
    simulations = simulatePCA(nSim, spearman_corr_matrix)
    simulations = pd.DataFrame(simulations.T, columns=[stock for stock in portfolio["Stock"]])
    
    # Transform the simulations into uniform variables using standard normal CDF.
    uni = norm.cdf(simulations)
    uni = pd.DataFrame(uni, columns=[stock for stock in portfolio["Stock"]])
    
    simulatedReturns = pd.DataFrame()
    # Transform the uniform variables into the desired data using quantile.
    for stock in portfolio["Stock"]:
        # If the distribution for the model is normal, use the quantile of the normal distribution.
        if portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'Normal':
            mu, sigma = models[stock]
            simulatedReturns[stock] = norm.ppf(uni[stock], loc=mu, scale=sigma)
            
        # If the distribution for the model is t, use the quantile of the t distribution.
        elif portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'T':
            nu, mu, sigma = models[stock]
            simulatedReturns[stock] = t.ppf(uni[stock], df=nu, loc=mu, scale=sigma)
    
    simulatedValue = pd.DataFrame()
    pnl = pd.DataFrame()
    # Calculate the daily prices for each stock
    for stock in portfolio["Stock"]:
        currentValue = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        simulatedValue[stock] = currentValue * (1 + simulatedReturns[stock])
        pnl[stock] = simulatedValue[stock] - currentValue
        
    risk = pd.DataFrame(columns = ["Stock", "VaR95", "ES95", "VaR95_Pct", "ES95_Pct"])
    w = pd.DataFrame()

    for stock in pnl.columns:
        i = risk.shape[0]
        risk.loc[i, "Stock"] = stock
        risk.loc[i, "VaR95"] = -np.percentile(pnl[stock], 5)
        risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        risk.loc[i, "ES95"] = -pnl[stock][pnl[stock] <= -risk.loc[i, "VaR95"]].mean()
        risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        
        # Determine the weights for the two stock
        w.at['Weight', stock] = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0] / portfolio['CurrentValue'].sum()
        
    # Calculate the total pnl.
    pnl['Total'] = 0
    for stock in portfolio["Stock"]:
        pnl['Total'] += pnl[stock]
    
    i = risk.shape[0]
    risk.loc[i, "Stock"] = 'Total'
    risk.loc[i, "VaR95"] = -np.percentile(pnl['Total'], 5)
    risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio['CurrentValue'].sum()
    risk.loc[i, "ES95"] = -pnl['Total'][pnl['Total'] <= -risk.loc[i, "VaR95"]].mean()
    risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio['CurrentValue'].sum()

    return risk

In [3]:
# Read the price data from csv files.
prices = pd.read_csv('DailyPrices.csv')

# Calculate the arithmetic returns from prices.
returns = return_calculate(prices)

# Center the data.
returns -= returns.mean()

# Read the portfolio data from csv files.
portfolio = pd.read_csv('portfolio.csv')

# Assign 'T' to portfolio A and B, and 'Normal' to portfolio C
portfolio.loc[portfolio['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio.loc[portfolio['Portfolio'] == 'C', 'Distribution'] = 'Normal'

for stock in portfolio["Stock"]:
    portfolio.loc[portfolio['Stock'] == stock, 'Starting Price'] = prices.iloc[-1][stock]

# Fit Generalized T models to stocks in portfolios A and B, and fit a normal distributions to stocks 
# in portfolio C. Calculate the VaR and ES of each portfolio as well as your total VaR and ES. You 
# will need to use a copula.  Compare the results from this to your VaR form Problem 3 from 
# Week 4.
simulateCopula(portfolio, returns)

,Stock,VaR95,ES95,VaR95_Pct,ES95_Pct
0,AAPL,316.770197,414.719498,0.036256,0.047466
1,TSLA,142.316337,183.04995,0.068646,0.088293
2,JPM,266.365275,356.288227,0.029639,0.039645
3,HD,266.378288,368.138502,0.031266,0.04321
4,BAC,246.743411,346.549801,0.033051,0.04642
...,...,...,...,...,...
95,LRCX,401.329831,499.319947,0.055607,0.069184
96,MO,241.230168,301.101612,0.02643,0.03299
97,LMT,342.535399,427.567205,0.026987,0.033687
98,TFC,242.312036,304.100173,0.03393,0.042582


In [4]:
portfolio_a = portfolio.loc[portfolio["Portfolio"] == "A"]
risk_a = simulateCopula(portfolio_a, returns)
print(risk_a)
portfolio_b = portfolio.loc[portfolio["Portfolio"] == "B"]
risk_b = simulateCopula(portfolio_b, returns)
print(risk_b)
portfolio_c = portfolio.loc[portfolio["Portfolio"] == "C"]
risk_c = simulateCopula(portfolio_c, returns)
print(risk_c)

/var/folders/jf/47sxrbks7kq5ksg3d4jdsvlc0000gn/T/ipykernel_14446/1745023637.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']


    Stock        VaR95          ES95 VaR95_Pct  ES95_Pct
0    AAPL   318.582656    415.072359  0.036463  0.047507
1    TSLA   144.073323    184.048754  0.069493  0.088775
2     JPM   269.858786    358.051258  0.030028  0.039842
3      HD   258.909286    367.478831  0.030389  0.043132
4     BAC    247.77006    337.707824  0.033189  0.045236
5     XOM   557.151826    740.129779  0.034802  0.046232
6    AVGO   384.109068    498.306164  0.037695  0.048902
7     PEP    192.70677     284.27346  0.019567  0.028865
8     TMO   323.357118    437.488706  0.033433  0.045233
9   CMCSA   217.926236    310.467746  0.029639  0.042226
10   META   339.664595    506.262432  0.057851  0.086226
11    ACN   271.437398    363.792917  0.032823  0.043991
12   INTC   196.268398    270.279565  0.039321  0.054149
13   PYPL   249.448725    332.450888  0.055806  0.074375
14    MRK   264.597246    380.695595  0.020157  0.029002
15      T   178.609101    259.755011  0.025585  0.037209
16    LOW   295.360227    396.2

/var/folders/jf/47sxrbks7kq5ksg3d4jdsvlc0000gn/T/ipykernel_14446/1745023637.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']


    Stock        VaR95         ES95 VaR95_Pct  ES95_Pct
0    MSFT   331.844614   436.502065  0.038145  0.050176
1   GOOGL    16.107943     21.75694  0.042385  0.057249
2    NVDA   569.260496    709.01742  0.067066  0.083531
3     JNJ   176.708296   242.713666  0.018247  0.025063
4      PG   192.149578   274.000421  0.022255  0.031734
5      MA   326.811789   453.010045  0.032677  0.045295
6     DIS   276.050869   368.853113    0.0379   0.05064
7    ADBE   326.070012   465.202256  0.043382  0.061893
8      KO   189.279531   275.483517  0.019477  0.028348
9    NFLX   441.777265   630.240201  0.060935   0.08693
10   COST   294.884741   477.077905  0.029451  0.047648
11    WFC   283.137419   401.226102  0.034405  0.048754
12    WMT   235.900642   368.539734  0.024158  0.037741
13    LLY   394.251653   530.660042  0.028134  0.037868
14    NKE   355.153949   502.974558  0.042747  0.060539
15    LIN   320.788752   423.704645  0.031115  0.041098
16    UNP   230.567507   304.227216  0.027728  0

/var/folders/jf/47sxrbks7kq5ksg3d4jdsvlc0000gn/T/ipykernel_14446/1745023637.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']


    Stock        VaR95         ES95 VaR95_Pct  ES95_Pct
0    AMZN    20.373554    25.272985  0.051846  0.064314
1    GOOG    16.281739    20.006458   0.04264  0.052395
2   BRK-B   226.461988   284.308555  0.023779  0.029853
3     UNH    276.13302   344.706592   0.02584  0.032258
4       V   326.108994   409.909763  0.030253  0.038027
5     PFE   221.682643   279.879973  0.028104  0.035482
6    CSCO   230.690648   288.040203  0.030286  0.037815
7     CVX   465.169865   576.187736  0.035406  0.043856
8    ABBV    263.94499   329.574092  0.023987  0.029951
9     ABT   235.747298   295.094043  0.027613  0.034564
10    CRM   372.504283   470.298659   0.04875  0.061549
11     VZ   178.353977   222.186697   0.02383  0.029687
12   QCOM    342.90921    426.23744  0.049567  0.061612
13    MCD   218.051891   269.497088   0.02145   0.02651
14    DHR   318.176738   393.566087  0.035635  0.044078
15    TXN    309.81049   386.708583  0.032786  0.040924
16     PM   258.649331   321.349284  0.026137  0

In [9]:
x = 8141.803692+6970.138687+5976.135057
y = 10633.076615+9244.929368+7526.456653
print(x)
print(y)

21088.077436
27404.462636
